In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

"""
Load the dataset.
X will have the features.
Y will have the label we want to predict.
"""
df = pd.read_csv("../../data/heartdiseasetrain/heart.csv")
X = df.drop("target", axis=1)
y = df["target"]

num_features = X.shape[1]

"""
GA Tuning Parameters:
- POP_SIZE: Number of candidate solutions (feature subsets) in each generation.
- NUM_GENERATIONS: How many iterations the GA will run to evolve better feature subsets.
- MUTATION_RATE: Probability of randomly flipping a bit in the feature mask to maintain diversity.
"""
POP_SIZE = 30
NUM_GENERATIONS = 50
MUTATION_RATE = 0.1

"""
This function is the heart of the GA.
The selected variable is a list of indices of features that are included in the current subset (where bit == 1).
We split the data into training and testing sets using only the selected features, train a Decision Tree classifier, and return the accuracy on the test set as the fitness score. If no features are selected, we return a fitness of 0 to avoid evaluating an empty feature set.
"""
def fitness(individual):
    selected = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected) == 0:
        return 0  # avoid empty feature sets

    X_train, X_test, y_train, y_test = train_test_split(
        X.iloc[:, selected], y, test_size=0.3, random_state=42
    )

    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

"""
This function creates a population of random features.
"""
def init_population():
    return np.random.randint(0, 2, (POP_SIZE, num_features))

"""
Pick two individuals at random from the population and return the one with the higher fitness.
This is what makes the GA evolve to the better solution.
"""
def select(pop, fitnesses):
    idx1, idx2 = np.random.randint(0, POP_SIZE, 2)
    return pop[idx1] if fitnesses[idx1] > fitnesses[idx2] else pop[idx2]

"""
Combine two parents by picking a random location to split the features, then take the 'left'
side features of one and combine with the 'right' side of the other.
"""
def crossover(parent1, parent2):
    point = np.random.randint(1, num_features - 1)
    child = np.concatenate([parent1[:point], parent2[point:]])
    return child

"""
This mutates the given indivual by randomly flipping bits with a certain probability (MUTATION_RATE). This helps maintain diversity in the population and allows the GA to explore new feature combinations that may not be reached through selection and crossover alone.
"""
def mutate(individual):
    for i in range(num_features):
        if np.random.rand() < MUTATION_RATE:
            individual[i] = 1 - individual[i]
    return individual

"""
Run the GA
"""
population = init_population()

for gen in range(NUM_GENERATIONS):
    fitnesses = np.array([fitness(ind) for ind in population])

    new_population = []
    for _ in range(POP_SIZE):
        parent1 = select(population, fitnesses)
        parent2 = select(population, fitnesses)
        child = crossover(parent1, parent2)
        child = mutate(child)
        new_population.append(child)

    population = np.array(new_population)

    best_idx = np.argmax(fitnesses)
    print(f"Generation {gen+1}: Best Accuracy = {fitnesses[best_idx]:.4f}")

"""
Show the best features identified by GA.
"""
best_individual = population[np.argmax([fitness(ind) for ind in population])]
selected_features = [col for col, bit in zip(X.columns, best_individual) if bit == 1]

print("\nBest feature subset:")
print(selected_features)
print(f"Generation {gen+1}: Best Accuracy = {fitnesses[best_idx]:.4f}")


Generation 1: Best Accuracy = 0.7692
Generation 2: Best Accuracy = 0.8022
Generation 3: Best Accuracy = 0.8352
Generation 4: Best Accuracy = 0.8242
Generation 5: Best Accuracy = 0.8352
Generation 6: Best Accuracy = 0.8132
Generation 7: Best Accuracy = 0.8242
Generation 8: Best Accuracy = 0.8352
Generation 9: Best Accuracy = 0.8242
Generation 10: Best Accuracy = 0.8242
Generation 11: Best Accuracy = 0.8242
Generation 12: Best Accuracy = 0.8242
Generation 13: Best Accuracy = 0.8022
Generation 14: Best Accuracy = 0.8022
Generation 15: Best Accuracy = 0.8352
Generation 16: Best Accuracy = 0.8462
Generation 17: Best Accuracy = 0.8022
Generation 18: Best Accuracy = 0.8242
Generation 19: Best Accuracy = 0.8022
Generation 20: Best Accuracy = 0.8462
Generation 21: Best Accuracy = 0.8242
Generation 22: Best Accuracy = 0.8242
Generation 23: Best Accuracy = 0.8242
Generation 24: Best Accuracy = 0.8242
Generation 25: Best Accuracy = 0.8242
Generation 26: Best Accuracy = 0.8352
Generation 27: Best A